In [1]:
# To default to float division
from __future__ import division

# Core python libraries
import csv
import re
import math
import collections

# External libraries.
import nltk
import numpy
import pandas
import scipy
import sklearn
# Sklearn is not eager in importing all of its subpackages. Have to explicity load the ones we want.
import sklearn.ensemble

# So you know when this code block finishes.
print ("Done")

Done


In [2]:
LDA = False

data = pandas.read_csv('data/train.csv', nrows=10000)

X_train_index, X_test_index, Y_train, Y_test = sklearn.cross_validation.train_test_split(
    data.index, data['LogSalaryNormalized'], test_size=.33, random_state=42)

# Keep train and test as pandas dataframes.
X_train = data.iloc[X_train_index]
X_test = data.iloc[X_test_index]

if LDA:
    lda_data = pandas.read_csv('data/train_large_lda_50.csv')

    X_train_lda = lda_data.iloc[X_train_index]
    X_test_lda = lda_data.iloc[X_test_index]


data.head()


,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName,TitleRaw,DescriptionLength,LogSalaryNormalized,FullDescriptionWithTitle
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,unknown_value,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk,Engineering Systems Analyst,348,10.126631,Engineering Systems Analyst Engineering System...
1,12613049,Engineering Systems Analyst Mathematical Modeller,Engineering Systems Analyst / Mathematical Mod...,"Surrey, South East, South East",Surrey,unknown_value,permanent,Gregory Martin International,Engineering Jobs,25000 - 30000/annum 25K-30K negotiable,27500,cv-library.co.uk,Engineering Systems Analyst / Mathematical Mod...,397,10.221941,Engineering Systems Analyst Mathematical Model...
2,12613647,Pioneer Miser Engineering Systems Analyst,"Pioneer, Miser Engineering Systems Analyst Do...","Surrey, South East, South East",Surrey,unknown_value,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk,"Pioneer, Miser Engineering Systems Analyst",216,10.126631,Pioneer Miser Engineering Systems Analyst Pion...
3,13179816,Engineering Systems Analyst Water Industry,Engineering Systems Analyst Water Industry Loc...,"Dorking, Surrey, Surrey, Surrey",Dorking,unknown_value,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20K to 30K,25000,cv-library.co.uk,Engineering Systems Analyst Water Industry,1602,10.126631,Engineering Systems Analyst Water Industry Eng...
4,14131336,Senior Subsea Pipeline Integrity Engineer,A globally renowned engineering and training c...,"Aberdeen, Borders",UK,unknown_value,permanent,Indigo 21 Ltd,Engineering Jobs,50000 - 100000/annum,75000,cv-library.co.uk,Senior Subsea Pipeline Integrity Engineer,1364,11.225243,Senior Subsea Pipeline Integrity Engineer A gl...


In [47]:
# All the words
MIN_WORD_FREQUENCY = 1

# Creates a true word count
count_vect = sklearn.feature_extraction.text.CountVectorizer(
  stop_words='english', min_df=MIN_WORD_FREQUENCY)

# Creates a hashed count
hash_vect = sklearn.feature_extraction.text.HashingVectorizer(
  stop_words='english', n_features=10000)



# Here I am count vectoring the full description field. But in theory
# any text field can be processed this way.
X_train_full_description_counts = count_vect.fit_transform(X_train['FullDescription'])
X_test_full_description_counts = count_vect.transform(X_test['FullDescription'])

# Here I am hash vectoring the full description field. But in theory
# any text field can be processed this way.
X_train_full_description_hashs = hash_vect.fit_transform(X_train['FullDescription'])
X_test_full_description_hashs = hash_vect.transform(X_test['FullDescription'])

print(X_train_full_description_counts.shape)
print(X_test_full_description_counts.shape)
#TODO (max): supress the VisibleDeprecationWarning here from numpy. It is being thrown by the package.

print(X_train_full_description_hashs.shape)
print(X_test_full_description_hashs.shape)

(16750, 42132)
(8250, 42132)
(16750, 10000)
(8250, 10000)


In [4]:
new_column = data['Title'].str.replace(r'Excellent?', r'Excellent')
new_column = new_column.str.replace(r'Operationa(al)?', r'Operational')
new_column = new_column.str.replace(r'Switzerla(nd)?', r'Switzerland')
new_column = new_column.str.replace(r'progammer', r'programmer')
new_column = new_column.str.replace(r'Progamming', r'Programming')
new_column = new_column.str.replace(r'Registerd', r'Registered')
new_column = new_column.str.replace(r'Technician?', r'Technician')




new_column = new_column.str.replace(r'Develpoer', r'Developer')
new_column = new_column.str.replace(r'Administraotr|Administartor', r'Administrator')
new_column = new_column.str.replace(r'Restauarnt', r'Restauarnt')
new_column = new_column.str.replace(r'Eneigneer', r'Engineer')


new_column = new_column.str.replace(r'(\d+) %', r'\1%')
new_column = new_column.str.replace(r'(\w+)(\d+%)', r'\1 \2')
# Replace some 10%bonus with 10% bonus
new_column = new_column.str.replace(r'(\d+%)(\w+)', r'\1 \2')

new_column = new_column.str.replace(r'FinanceCheshire', r'Finance Cheshire')
new_column = new_column.str.replace(r'ManufacturingCoventry', r'Manufacturing Coventry')


new_column = new_column.str.replace(r'(?i)(Manager)(\w+)', r'\1 \2')
# Multi words concated can be split by their capitals. #Does not work
new_column = new_column.str.replace(r'([a-z]+)([A-Z][a-z]+)', r'\1 \2')




#X_train['Title'] = X_train['Title']
job_title_words = collections.Counter()
for title in new_column:
    for word in title.split(' '):
        job_title_words[word] +=1

for word, count in job_title_words.most_common():
    print "%d   %s" % (count, word) 


34008   Manager
19467   ****
16337   Engineer
12582   Sales
11555   Senior
9285   Developer
8049   Assistant
6842   Executive
6690   Business
6402   Analyst
6274   Consultant
6211   London
5671   
5629   Support
5312   and
5222   k
4900   Project
4847   Nurse
4623   Development
4416   Account
4271   Service
4216   Administrator
3746   Software
3735   &
3505   Teacher
3320   Marketing
3230   Care
3204   Design
3063   Recruitment
3040   Chef
2989   C
2978   in
2976   Advisor
2886   Technical
2801   of
2698   IT
2620   Accountant
2613   Worker
2592   Customer
2495   K
2473   Head
2466   to
2435   Officer
2383   Technician
2342   Finance
2334   Team
2326   Mechanical
2322   Home
2272   RGN
2242   Financial
2228   Web
2180   Services
2135   Systems
2127   Management
2084   Graduate
2065   Lead
1987   Java
1969   HR
1957   Director
1936   SQL
1924   Coordinator
1913   Commercial
1757   Electrical
1747   Quality
1689   General
1667   Digital
1661   Specialist
1650   Leader
1648   Maintenance


In [ ]:
def original_salary_scale(log_value):
    return math.exp(log_value)

def mean_absolute_error_salary_scale(y_test, y_predicted):
    return sklearn.metrics.mean_absolute_error(
        numpy.exp(y_test), numpy.exp(y_predicted))

# Guess the average. Create an empty vector of the desired shape.
average_guess = numpy.empty(Y_test.shape)
average_guess.fill(numpy.mean(Y_train))

mean_guess = average_guess[0]
print('Mean salary value in training set is  £{:10.2f}'.format(original_salary_scale(mean_guess)))
average_guess_mae = mean_absolute_error_salary_scale(Y_test, average_guess)
print('Guess the average Mean Absolute Error: {:10.4f}'.format(average_guess_mae))

# SGD Needs normalized inputs
# TODO: Suppress DataConversionWarning from converting ints to floats here.
normalizer = sklearn.preprocessing.Normalizer(norm='l1')
X_train_norm = normalizer.fit_transform(X_train_full_description_counts.astype('float64'))
X_test_norm = normalizer.transform(X_test_full_description_counts.astype('float64'))

# We want a stochastic gradient descent with l1 norm.
sgd = sklearn.linear_model.SGDRegressor(alpha=.005, penalty='l1', n_iter=100)
sgd.fit(X_train_norm, Y_train)
sgd_predictions = sgd.predict(X_test_norm)
sgd_mae = mean_absolute_error_salary_scale(Y_test, sgd_predictions)
print('SGDRegressor Mean Absolute Error: {:10.4f}'.format(sgd_mae))
#TODO (any): wonder why this is so inaccurate/wrong

# Random Forest.
rf = sklearn.ensemble.RandomForestRegressor(n_estimators=10)
rf.fit(X_train_full_description_counts, Y_train)
rf_predictions = rf.predict(X_test_full_description_counts)

# Mean Absolute Error
rf_mae = mean_absolute_error_salary_scale(Y_test, rf_predictions)
print('Random Forest Regressor Mean Absolute Error: {:10.4f}'.format(rf_mae))

In [9]:
print(sgd.intercept_)
print(mean_guess)

[ 10.24537985]
10.2453808753


In [22]:
rf = sklearn.ensemble.RandomForestRegressor(n_estimators=100, n_jobs=-1, warm_start=True)

In [ ]:
# If warm start is set we can add more estimators to our random forest
print("We currently have %s trees in our random forest" % len(rf.estimators_))

rf.n_estimators +=2000
if LDA:
    # Fit some models on the LDA output data to see how they perform.
    # Random Forest.
    rf.fit(X_train_lda, Y_train)
    rf_predictions = rf.predict(X_test_lda)

    # Mean Absolute Error
    rf_mae = mean_absolute_error_salary_scale(Y_test, rf_predictions)
    print('Random Forest Regressor Mean Absolute Error: {:10.4f}'.format(rf_mae))

We currently have 2250 trees in our random forest
